In [1]:
import numpy as np 
import pandas as pd
import requests
from groq import Groq
from datetime import datetime
import calendar
import matplotlib.pyplot as plt
import math
from openai import OpenAI
import time

In [127]:
contracts = pd.read_csv("/home/xikron/Projects/misc/data/pC.csv")
contracts['Start Date']= pd.to_datetime(contracts['Start Date'])
contracts=contracts.drop_duplicates(subset='Award ID').sample(100, replace=False).sort_values(by='Start Date')
contracts.head()

,internal_id,Award ID,Recipient Name,Award Amount,Total Outlays,Description,Contract Award Type,def_codes,COVID-19 Obligations,COVID-19 Outlays,...,Infrastructure Outlays,Awarding Agency,Awarding Sub Agency,Start Date,End Date,recipient_id,prime_award_recipient_id,awarding_agency_id,agency_slug,generated_internal_id
263,15133929,FC09,GD,3.270593e+07,NaN,INFORMATION TECHNOLOGY (IT) SUPPORT,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Defense,Department of the Navy,2010-01-29,2013-03-01,01bd8490-759e-06b2-8d0e-fe788c9107fd-C,NaN,1173.0,department-of-defense,CONT_AWD_FC09_9700_N0017804D4030_9700
119,25887346,N0002410C2203,HII,2.367899e+08,NaN,LPD 17 CLASS ENGINEERING SERVICES,DEFINITIVE CONTRACT,['Q'],NaN,NaN,...,NaN,Department of Defense,Department of the Navy,2010-02-10,2016-03-31,1eb5d9d6-aabf-3228-e6ab-79cf9729c9d7-C,NaN,1173.0,department-of-defense,CONT_AWD_N0002410C2203_9700_-NONE-_-NONE-
149,15093891,FA880710C0001,RTN,4.148269e+09,9.016155e+07,GPS ADVANCED CONTROL SEGMENT (OCX) PHASE B BLO...,DEFINITIVE CONTRACT,['Q'],NaN,NaN,...,NaN,Department of Defense,Department of the Air Force,2010-02-25,2024-12-31,988be7bc-cdfb-c4c1-87ad-3f6d251f7392-C,NaN,1173.0,department-of-defense,CONT_AWD_FA880710C0001_9700_-NONE-_-NONE-
38,30534599,SPM1C110C0011,BAESY,2.797354e+07,NaN,"4513600332!HELMET,GROUND TROOP",DEFINITIVE CONTRACT,NaN,NaN,NaN,...,NaN,Department of Defense,Defense Logistics Agency,2010-03-09,2010-10-05,585b1232-bff3-7bf0-4fd1-53ea5d864724-C,NaN,1173.0,department-of-defense,CONT_AWD_SPM1C110C0011_9700_-NONE-_-NONE-
312,13790570,DTFAWA10C00052,LMT,4.458355e+08,1.099002e+08,WA-10-03770-- INCREMENTAL FUNDING - NEXTGEN H...,DEFINITIVE CONTRACT,['Q'],NaN,NaN,...,NaN,Department of Transportation,Federal Aviation Administration,2010-05-03,2024-11-01,81b6aa7b-b2d5-62f3-89db-70fbede89173-C,NaN,731.0,department-of-transportation,CONT_AWD_DTFAWA10C00052_6920_-NONE-_-NONE-


In [128]:
contracts.shape

(100, 21)

In [129]:
capital = 10000000
portfolio = {}
valuation = []

In [130]:
def setLink (ticker, start, end, freq):
    link="https://financialmodelingprep.com/api/v3/historical-chart/"+freq+"/"+ticker+"?from="+start+"&to="+end+"&apikey=26srycwxWrFIhEuaZwic6mBdx7f4VjGT"
    return link

In [118]:
response = requests.get(url=setLink("LMT", "2010-03-31", "2010-03-31", "5min"))
response.json()

[{'date': '2010-03-31 15:55:00',
  'open': 83.2,
  'low': 83.18,
  'high': 83.3,
  'close': 83.24,
  'volume': 115324},
 {'date': '2010-03-31 15:50:00',
  'open': 83.08,
  'low': 83.06,
  'high': 83.275,
  'close': 83.2,
  'volume': 102237},
 {'date': '2010-03-31 15:45:00',
  'open': 83.22,
  'low': 83.05,
  'high': 83.24,
  'close': 83.09,
  'volume': 102534},
 {'date': '2010-03-31 15:40:00',
  'open': 83.315,
  'low': 83.17,
  'high': 83.33,
  'close': 83.24,
  'volume': 67305},
 {'date': '2010-03-31 15:35:00',
  'open': 83.39,
  'low': 83.27,
  'high': 83.39,
  'close': 83.31,
  'volume': 50490},
 {'date': '2010-03-31 15:30:00',
  'open': 83.49,
  'low': 83.36,
  'high': 83.51,
  'close': 83.39,
  'volume': 30398},
 {'date': '2010-03-31 15:25:00',
  'open': 83.34,
  'low': 83.33,
  'high': 83.49,
  'close': 83.49,
  'volume': 33612},
 {'date': '2010-03-31 15:20:00',
  'open': 83.41,
  'low': 83.32,
  'high': 83.43,
  'close': 83.32,
  'volume': 22938},
 {'date': '2010-03-31 15:15:00

In [119]:
len(response.json())

78

In [131]:
from dataclasses import dataclass

@dataclass
class transaction:
    contractID: int
    num: int

In [136]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

def getBuyDay(contractDate):
    uBday = CustomBusinessDay(calendar=USFederalHolidayCalendar())
    contractDate = pd.Timestamp(contractDate)
    buyDay = contractDate - 3 * uBday
    return buyDay.strftime('%Y-%m-%d')

def getSellDay(contractDate):
    uBday = CustomBusinessDay(calendar=USFederalHolidayCalendar())
    contractDate = pd.Timestamp(contractDate)
    sellDay = contractDate + 4*uBday
    return sellDay.strftime('%Y-%m-%d')

In [133]:
def buy(ticker, valuation, buyDate, id): 
    global portfolio, capital
    response = requests.get(url=setLink(ticker, buyDate, buyDate, "5min"))
    if response.status_code == 200 and len(response.json()) >15: 
        pps = response.json()[-15]["open"]
        shares = math.floor(valuation/pps)
        if ticker not in portfolio: 
            portfolio[ticker]= [transaction(contractID=id, num=shares)] 
        else: 
            portfolio[ticker].append(transaction(contractID=id, num=shares))
        capital-=shares*pps 
        print("purchased "+ str(shares) + " shares of "+ticker+" for "+ str(shares*pps)+". current portfolio valued at "+ str(capital))
        

In [134]:
def sell(ticker, sellDate, id):
    global portfolio, capital, valuation
    response = requests.get(url=setLink(ticker, sellDate, sellDate, "5min"))
    if response.status_code == 200 and len(response.json()) >15: 
        pps = response.json()[-15]["open"]
        for t in portfolio[ticker]: 
            if t.contractID == id: 
                shares = t.num
                value = shares*pps
                capital+=value 
                portfolio[ticker].remove(t)
                valuation.append(capital)
                print("sold "+str(shares) +" shares of " + ticker + " for " + str(value)+". current portfolio is valued at " + str(capital))
                break

In [ ]:
global capital

for i in range(200): 
    buy(contracts.iloc[i]['Recipient Name'], capital*0.1, getBuyDay(contracts.iloc[i]['Start Date']), contracts.iloc[i]['internal_id'])
    sell(contracts.iloc[i]['Recipient Name'], getSellDay(contracts.iloc[i]['Start Date']), contracts.iloc[i]['internal_id'])

purchased 1475 shares of GD for 99990.25000000001. current portfolio valued at 9900009.75
sold 1475 shares of GD for 103235.24999999999. current portfolio is valued at 10003245.0
purchased 1788 shares of RTN for 100020.72. current portfolio valued at 9903224.28
sold 1788 shares of RTN for 101522.64. current portfolio is valued at 10004746.92
purchased 1182 shares of LMT for 99973.56. current portfolio valued at 9904773.36
sold 1182 shares of LMT for 100824.59999999999. current portfolio is valued at 10005597.959999999
purchased 1734 shares of NOC for 99999.78. current portfolio valued at 9905598.18
sold 1734 shares of NOC for 99115.43999999999. current portfolio is valued at 10004713.62
purchased 2290 shares of CACI for 100004.3. current portfolio valued at 9904709.319999998
sold 2290 shares of CACI for 101905.0. current portfolio is valued at 10006614.319999998
purchased 2303 shares of CACI for 100065.35. current portfolio valued at 9906548.969999999
sold 2303 shares of CACI for 10446

KeyError: 'BLH'